# Operaciones Avanzadas: Pandas vs Polars vs Data.table

Este notebook se centra en operaciones avanzadas y transformaciones complejas de datos para comparar el rendimiento de las tres librerías.

## Contenido
1. Configuración y Preparación
2. Joins Complejos
3. Window Functions
4. Operaciones de Pivote
5. Agregaciones Complejas

In [ ]:
import pandas as pd
import polars as pl
import datatable as dt
import numpy as np
from time import time
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('..')
from src.utils import plot_benchmark_results, calculate_speedup
from src.benchmarks import create_sample_data

## 1. Preparación de Datos Complejos

In [ ]:
# Crear datasets más complejos para operaciones avanzadas
def create_complex_datasets(n_rows: int):
    # Dataset principal
    main_data = {
        'id': np.arange(n_rows),
        'value': np.random.randn(n_rows),
        'category': np.random.choice(['A', 'B', 'C'], n_rows),
        'date': pd.date_range('2023-01-01', periods=n_rows),
        'group': np.random.choice(['X', 'Y', 'Z'], n_rows)
    }
    
    # Dataset secundario para joins
    secondary_data = {
        'id': np.random.choice(np.arange(n_rows), size=n_rows//2),
        'aux_value': np.random.randn(n_rows//2),
        'aux_category': np.random.choice(['P', 'Q', 'R'], n_rows//2)
    }
    
    return main_data, secondary_data

N_ROWS = 1_000_000
main_data, secondary_data = create_complex_datasets(N_ROWS)

## 2. Joins Complejos

In [ ]:
def benchmark_joins():
    results = {}
    
    # Pandas
    start = time()
    df1_pd = pd.DataFrame(main_data)
    df2_pd = pd.DataFrame(secondary_data)
    result_pd = df1_pd.merge(df2_pd, on='id', how='left')
    results['pandas'] = time() - start
    
    # Polars
    start = time()
    df1_pl = pl.DataFrame(main_data)
    df2_pl = pl.DataFrame(secondary_data)
    result_pl = df1_pl.join(df2_pl, on='id', how='left')
    results['polars'] = time() - start
    
    # Data.table
    start = time()
    df1_dt = dt.Frame(main_data)
    df2_dt = dt.Frame(secondary_data)
    result_dt = df1_dt[:, :, dt.join(df2_dt)]
    results['datatable'] = time() - start
    
    return results

join_results = benchmark_joins()
plot_benchmark_results(join_results, title='Tiempo de Joins')

## 3. Window Functions

In [ ]:
def benchmark_window_operations():
    results = {}
    
    # Pandas
    start = time()
    df_pd = pd.DataFrame(main_data)
    result_pd = df_pd.assign(
        rolling_mean=df_pd.groupby('category')['value'].transform(
            lambda x: x.rolling(window=7, min_periods=1).mean()
        ),
        cumulative_sum=df_pd.groupby('category')['value'].transform('cumsum')
    )
    results['pandas'] = time() - start
    
    # Polars
    start = time()
    df_pl = pl.DataFrame(main_data)
    result_pl = df_pl.with_columns([
        pl.col('value').rolling_mean(7).over('category').alias('rolling_mean'),
        pl.col('value').cum_sum().over('category').alias('cumulative_sum')
    ])
    results['polars'] = time() - start
    
    # Data.table
    start = time()
    df_dt = dt.Frame(main_data)
    result_dt = df_dt[:, {
        'rolling_mean': dt.roll_mean(dt.f.value, window=7),
        'cumulative_sum': dt.cumsum(dt.f.value)
    }, by('category')]
    results['datatable'] = time() - start
    
    return results

window_results = benchmark_window_operations()
plot_benchmark_results(window_results, title='Tiempo de Operaciones Window')

## 4. Operaciones de Pivote

In [ ]:
def benchmark_pivot_operations():
    results = {}
    
    # Pandas
    start = time()
    df_pd = pd.DataFrame(main_data)
    pivot_pd = df_pd.pivot_table(
        values='value',
        index='category',
        columns='group',
        aggfunc=['mean', 'count', 'std']
    )
    results['pandas'] = time() - start
    
    # Polars
    start = time()
    df_pl = pl.DataFrame(main_data)
    pivot_pl = df_pl.pivot(
        values='value',
        index='category',
        columns='group',
        aggregate_function=['mean', 'count', 'std']
    )
    results['polars'] = time() - start
    
    # Data.table (simulación de pivot)
    start = time()
    df_dt = dt.Frame(main_data)
    pivot_dt = df_dt[:, {'mean': dt.mean(dt.f.value),
                         'count': dt.count(),
                         'std': dt.sd(dt.f.value)},
                    by('category,group')]
    results['datatable'] = time() - start
    
    return results

pivot_results = benchmark_pivot_operations()
plot_benchmark_results(pivot_results, title='Tiempo de Operaciones Pivot')

## 5. Agregaciones Complejas

In [ ]:
def benchmark_complex_aggregations():
    results = {}
    
    # Pandas
    start = time()
    df_pd = pd.DataFrame(main_data)
    agg_pd = df_pd.groupby(['category', 'group']).agg({
        'value': ['mean', 'std', 'count', 'sum',
                 lambda x: x.quantile(0.25),
                 lambda x: x.quantile(0.75)]
    }).reset_index()
    results['pandas'] = time() - start
    
    # Polars
    start = time()
    df_pl = pl.DataFrame(main_data)
    agg_pl = df_pl.groupby(['category', 'group']).agg([
        pl.col('value').mean(),
        pl.col('value').std(),
        pl.col('value').count(),
        pl.col('value').sum(),
        pl.col('value').quantile(0.25),
        pl.col('value').quantile(0.75)
    ])
    results['polars'] = time() - start
    
    # Data.table
    start = time()
    df_dt = dt.Frame(main_data)
    agg_dt = df_dt[:, {
        'mean': dt.mean(dt.f.value),
        'std': dt.sd(dt.f.value),
        'count': dt.count(),
        'sum': dt.sum(dt.f.value),
        'q25': dt.quantile(dt.f.value, 0.25),
        'q75': dt.quantile(dt.f.value, 0.75)
    }, by('category,group')]
    results['datatable'] = time() - start
    
    return results

agg_results = benchmark_complex_aggregations()
plot_benchmark_results(agg_results, title='Tiempo de Agregaciones Complejas')

## 6. Análisis de Resultados

Vamos a calcular los speedups relativos a Pandas para cada operación:

In [ ]:
all_results = {
    'Joins': join_results,
    'Window': window_results,
    'Pivot': pivot_results,
    'Aggregation': agg_results
}

speedups = {}
for operation, results in all_results.items():
    speedups[operation] = calculate_speedup(results)

speedup_df = pd.DataFrame(speedups)
print("\nSpeedup relativo a Pandas (>1 significa más rápido que Pandas):")
print(speedup_df)

# Visualizar speedups
plt.figure(figsize=(12, 6))
sns.heatmap(speedup_df, annot=True, fmt='.2f', cmap='YlOrRd')
plt.title('Speedup Relativo a Pandas')
plt.show()